<a href="https://colab.research.google.com/github/Roon311/YOLO/blob/main/Yolo_versions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://drive.google.com/uc?export=view&id=1QJ9PAT9q-Ksv_Vs_pLXtLHxjjV-9FMTz)



_Prepared by_  [**Noureldin Mohamed Abdelsalam**](mailto:s-noureldin.hamedo@zewailcity.edu.eg)

#**Linking Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Imports**

In [ ]:
import cv2
import time
import numpy as np
import torch

#**Yolov3**

In [ ]:
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
!pip install -r requirements.txt

Cloning into 'yolov3'...
remote: Enumerating objects: 10485, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 10485 (delta 237), reused 364 (delta 231), pack-reused 10078
Receiving objects: 100% (10485/10485), 9.73 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (7036/7036), done.
/content/yolov3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.5 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/yolo_project/
video="https://drive.google.com/file/d/1HGUgcEwFrhDjsyrOxYje9EIg1Gpdmhz3/view?usp=sharing"

/content/drive/MyDrive/yolo_project


In [ ]:
%ls

benchmarks.py    data/       LICENSE          requirements.txt  tutorial.ipynb
CITATION.cff     detect.py   models/          segment/          utils/
classify/        export.py   README.md        setup.cfg         val.py
CONTRIBUTING.md  hubconf.py  README.zh-CN.md  train.py


In [ ]:
model = torch.hub.load("ultralytics/yolov3", "yolov5l6")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov3_master
YOLOv3 🚀 2023-4-15 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 147M/147M [00:02<00:00, 52.3MB/s]

Fusing layers... 
YOLOv3l6 summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("/content/drive/MyDrive/yolo_project/yolov3.mp4", fourcc, video.get(cv2.CAP_PROP_FPS), (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
num_objects = 0
total_time = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    start_time = time.time()
    results = model(frame)
    end_time = time.time()
    detection_time = end_time - start_time
    total_time += detection_time
    num_objects += len(results.xyxy[0])

    #print(num_objects)
    #results.show()
    output_image=results.render()
    output_frame = np.array(output_image)[0]
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame)
video.release()
out.release()

##**Detection Time**

In [ ]:
print(total_time)

196.60236763954163


##**Total Number of objects detected**

In [ ]:
print(num_objects)

2437


After investigating this github repo, this is not yolov3, a problem occured with the crator and they did overwrite it with yolov5 so i will use darknet to apply yolov3

#**Using Darknet**

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.16 MiB | 16.57 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |

In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2023-04-16 12:45:40--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.7MB/s    in 12s     

2023-04-16 12:45:53 (19.1 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# run darknet detection
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg


 CUDA-version: 11080 (12000), cuDNN: 8.7.0, GPU count: 1  
 OpenCV version: 4.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
!python setup.py build_ext --inplace
!rm -rf build


Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/darknet/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.9/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darknet/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.9
creating build/temp.linux-x86_64-3.9/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -

In [ ]:
%cd /content/darknet/
start_time = time.time()
results=!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show "/content/drive/MyDrive/yolo_project/video.mp4" -out_filename "/content/drive/MyDrive/yolo_project/yolov3.mp4"
end_time = time.time()

/content/darknet


In [ ]:
import re

objects=[]
input_str=results[133:]
input_str = "".join(input_str)
while True:
  start = input_str.find("Objects:")
  end = input_str.find("FPS:", start)
  next = input_str.find("cvWriteFrame", start)
  objects_str = input_str[start:end].strip()
  objects.append(objects_str)
  input_str=input_str[next+1:]
  if objects_str== '':
    break
input_str = "".join(objects)
objects = input_str.replace("Objects:", "")
objects = re.sub(r'\d+%', '', objects)
objects = objects.replace(':', '')
objects = objects.split()
print(objects)

['car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person',

##**Detection Time GPU**

In [ ]:
total_time=end_time-start_time

In [ ]:
print(total_time)

11.288652658462524


##**Total Number of objects detected**

In [ ]:
print(len(objects),'objects')

3069 objects


#**Yolov5 models**.






In [ ]:
%cd  /content/
!git clone https://github.com/ultralytics/yolov5
%cd  /content/yolov5
!pip install -r requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15529, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 15529 (delta 49), reused 119 (delta 43), pack-reused 15393
Receiving objects: 100% (15529/15529), 14.59 MiB | 25.19 MiB/s, done.
Resolving deltas: 100% (10577/10577), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
def yolo_performer(modelname,video):
  model = torch.hub.load("ultralytics/yolov5", modelname)
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  print("creating output file")
  out = cv2.VideoWriter("/content/drive/MyDrive/yolo_project/"+modelname+".mp4", fourcc, video.get(cv2.CAP_PROP_FPS), (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
  print("output file creation done")
  num_objects = 0
  total_time = 0
  while True:
      ret, frame = video.read()
      if not ret:
          break
      start_time = time.time()
      results = model(frame)
      end_time = time.time()
      detection_time = end_time - start_time
      total_time += detection_time
      num_objects += len(results.xyxy[0])
      #print(num_objects)
      #results.show()
      output_image=results.render()
      output_frame = np.array(output_image)[0]
      output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
      out.write(output_frame)
  video.release()
  out.release()
  return num_objects,total_time


#  **yolov5n**  
---


In [ ]:
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
#yolov5n
yolov5n_num_objects,yolov5n_total_time=yolo_performer("yolov5n",video)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-16 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 3.87M/3.87M [00:00<00:00, 72.6MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


creating output file
output file creation done


In [ ]:
print(yolov5n_total_time,"seconds")
print(yolov5n_num_objects,"objects")

18.629981994628906 seconds
1906 objects


# **yolov5s**
---



In [ ]:
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
model = torch.hub.load("ultralytics/yolov5", "yolov5s")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("/content/drive/MyDrive/yolo_project/yolov5s.mp4", fourcc, video.get(cv2.CAP_PROP_FPS), (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
yolov5s_num_objects = 0
yolov5s_total_time = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    start_time = time.time()
    results = model(frame)
    end_time = time.time()
    detection_time = end_time - start_time
    yolov5s_total_time += detection_time
    yolov5s_num_objects += len(results.xyxy[0])
    #print(num_objects)
    #results.show()
    output_image=results.render()
    output_frame = np.array(output_image)[0]
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame)
video.release()
out.release()

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-16 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 51.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
print(yolov5s_total_time,"seconds")
print(yolov5s_num_objects,"objects")

45.31887912750244 seconds
2261 objects


#**yolov5m** 


In [ ]:
#yolov5m
yolov5m_num_objects,yolov5m_total_time=yolo_performer("yolov5m",video)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-15 Python-3.9.16 torch-2.0.0+cu118 CPU

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


creating output file
output file creation done


In [ ]:
print(yolov5m_total_time,"seconds")
print(yolov5m_num_objects,"objects")

132.85901141166687 seconds
2529 objects



#**yolov5l**  


In [ ]:
#yolov5l
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
model = torch.hub.load("ultralytics/yolov5", "yolov5l")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("/content/drive/MyDrive/yolo_project/yolov5l.mp4", fourcc, video.get(cv2.CAP_PROP_FPS), (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
yolov5l_num_objects = 0
yolov5l_total_time = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    start_time = time.time()
    results = model(frame)
    end_time = time.time()
    detection_time = end_time - start_time
    yolov5l_total_time += detection_time
    yolov5l_num_objects += len(results.xyxy[0])
    #print(num_objects)
    #results.show()
    output_image=results.render()
    output_frame = np.array(output_image)[0]
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame)
video.release()
out.release()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-15 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
print(yolov5l_total_time,"seconds")
print(yolov5l_num_objects,"objects")

255.8916175365448 seconds
2663 objects


#**yolov5x**
---

In [ ]:
#yolov5x
video = cv2.VideoCapture("/content/drive/MyDrive/yolo_project/video.mp4")
model = torch.hub.load("ultralytics/yolov5", "yolov5x")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("/content/drive/MyDrive/yolo_project/yolov5x.mp4", fourcc, video.get(cv2.CAP_PROP_FPS), (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
yolov5x_num_objects = 0
yolov5x_total_time = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    start_time = time.time()
    results = model(frame)
    end_time = time.time()
    detection_time = end_time - start_time
    yolov5x_total_time += detection_time
    yolov5x_num_objects += len(results.xyxy[0])
    #print(num_objects)
    #results.show()
    output_image=results.render()
    output_frame = np.array(output_image)[0]
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame)
video.release()
out.release()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-15 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
print(yolov5x_total_time,"seconds")
print(yolov5x_num_objects,"objects")

452.72636580467224 seconds
2613 objects


#**Latest YOLOv. (YOLOv8.0.78)**
---

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.78 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.2/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)
model.fuse()
start_time = time.time()
result=!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source="/content/drive/MyDrive/yolo_project/yolov8.mp4"
end_time = time.time()
yolov8_total_time=end_time-start_time


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [ ]:
result

['Ultralytics YOLOv8.0.78 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)',
 'YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs',
 '',
 'video 1/1 (1/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1 person, 3 cars, 70.4ms',
 'video 1/1 (2/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1 person, 3 cars, 62.6ms',
 'video 1/1 (3/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1 person, 3 cars, 62.6ms',
 'video 1/1 (4/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1 person, 3 cars, 1 traffic light, 62.6ms',
 'video 1/1 (5/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 2 persons, 3 cars, 36.4ms',
 'video 1/1 (6/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 2 persons, 3 cars, 37.1ms',
 'video 1/1 (7/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1 person, 2 cars, 37.5ms',
 'video 1/1 (8/145) /content/drive/MyDrive/yolo_project/yolov8.mp4: 448x640 1

In [ ]:
import re
results_cleaned=[]
total_count=0
for i in range(2,len(result)-1):
    start_index = result[i].find("448x640")
    last_comma_index = result[i].rfind(",")
    result_string = result[i][start_index+7:last_comma_index]
    int_numbers = [int(num) for num in re.findall(r'\b\d+\b', result_string)]
    int_numbers=np.array(int_numbers)
    results_cleaned=np.array(results_cleaned)
    results_cleaned=np.append(results_cleaned,int_numbers,axis=0)
    total_count+= sum(int_numbers)




In [ ]:
results_cleaned

array([          1,           3,           1,           3,           1,           3,           1,           3,           1,           2,           3,           2,           3,           1,           2,           1,           3,           1,           3,           2,           1,           3,           1,           4,
                 1,           4,           1,           5,           1,           5,           1,           1,           4,           1,           1,           4,           1,           4,           1,           5,           1,           5,           1,           1,           5,           1,           6,           1,
                 7,           1,           6,           1,           6,           6,           1,           4,           1,           5,           1,           1,           4,           1,           4,           1,           3,           1,           4,           1,           3,           1,           3,           1,
                 3,           1,           

In [ ]:
print("Total number of objects detected: ",total_count)

Total number of objects detected:  1481


In [ ]:
print("Total runtime on colab gpu",yolov8_total_time)

Total runtime on colab gpu 16.42167639732361


#**YOLO using ultralytics**

In [ ]:
import torch
import cv2
%pip install ultralytics
from ultralytics import YOLO 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 14.9 MB/s eta 0:00:00


In [ ]:
modelv5m = YOLO('yolov5mu.pt')
modelv3 = YOLO('yolov3u.pt')

100%|██████████| 48.2M/48.2M [00:00<00:00, 155MB/s]
100%|██████████| 198M/198M [00:00<00:00, 218MB/s]


#Yolov3

##cpu

In [ ]:
input_path = '/content/drive/MyDrive/yolo_project/video.mp4'
output_path = '/content/drive/MyDrive/yolo_project/ultralytics_results/yolov3.mp4'


cap = cv2.VideoCapture(input_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
encoder = cv2.VideoWriter(output_path, fourcc, 25, (frame_width, frame_height))  

time_taken = 0
total_detected_objects = 0

while 1 : 
    ret, frame = cap.read()
    if not ret :
      break

    result = modelv3.predict(frame, device = 'cpu')[0]

    im = cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB)  # get the labeled image
    encoder.write(im)

    time_taken +=  sum(result.speed.values())
    total_detected_objects += len(result.boxes.data)


cap.release()
encoder.release()
print(f'Total detected {total_detected_objects}, Total time {time_taken :.2f}')


0: 448x640 6 persons, 14 cars, 2 motorcycles, 1 traffic light, 12925.1ms
Speed: 3.8ms preprocess, 12925.1ms inference, 28.8ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 5 persons, 14 cars, 1 motorcycle, 1 traffic light, 9391.7ms
Speed: 0.9ms preprocess, 9391.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 14 cars, 2 motorcycles, 1 traffic light, 3618.9ms
Speed: 0.9ms preprocess, 3618.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 15 cars, 2 motorcycles, 1 traffic light, 4459.6ms
Speed: 0.8ms preprocess, 4459.6ms inference, 14.8ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 15 cars, 1 motorcycle, 1 traffic light, 6306.4ms
Speed: 0.9ms preprocess, 6306.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 7 persons, 15 cars, 1 traffic light, 4175.1ms
Speed: 0.9ms preprocess, 4175.1ms inference, 1.9ms postprocess per image at shap

Total detected 2614, Total time 509125.87


#Yolov5

##CPU

In [ ]:
input_path = '/content/drive/MyDrive/yolo_project/video.mp4'
output_path = '/content/drive/MyDrive/yolo_project/ultralytics_results/yolov3.mp4'


cap = cv2.VideoCapture(input_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
encoder = cv2.VideoWriter(output_path, fourcc, 25, (frame_width, frame_height))  

time_taken = 0
total_detected_objects = 0

while 1 : 
    ret, frame = cap.read()
    if not ret :
      break

    result = modelv5m.predict(frame, device = 'cpu')[0]

    im = cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB)  # get the labeled image
    encoder.write(im)

    time_taken +=  sum(result.speed.values())
    total_detected_objects += len(result.boxes.data)


cap.release()
encoder.release()
print(f'Total detected {total_detected_objects}, Total time {time_taken :.2f}')


0: 448x640 6 persons, 14 cars, 1 motorcycle, 771.9ms
Speed: 0.8ms preprocess, 771.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 14 cars, 1 motorcycle, 756.5ms
Speed: 0.9ms preprocess, 756.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 14 cars, 1 motorcycle, 746.0ms
Speed: 1.2ms preprocess, 746.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 16 cars, 1 motorcycle, 745.5ms
Speed: 0.9ms preprocess, 745.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 15 cars, 745.3ms
Speed: 0.8ms preprocess, 745.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 15 cars, 739.6ms
Speed: 0.8ms preprocess, 739.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 6 persons, 15 cars, 748.1ms
Speed: 0.8ms preprocess, 748.1ms inference, 1.3ms postprocess per image at s

Total detected 2583, Total time 123519.70
